In [ ]:
from nbdev import *
%nbdev_default_export functions

Cells will be exported to pct.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
#hide
#%load_ext autoreload
#%autoreload 2

In [ ]:
import numpy as np

# Functions

Functions that form the elements of a perceptual control node (system).

In [ ]:
%nbdev_export
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function."
    def __init__(self):
        self.output = np.zeros(1)
        self.links = []
        
    @abstractmethod
    def __call__(self):
        pass
    
    def set_output(self, output):
        self.output= output
    
    def get_output(self):
        return self.output
    
    def add_link(self, linkfn):
        self.links.append(linkfn)


In [ ]:
%nbdev_export
class Constant(BaseFunction):
    "A function that returns a constant value."
    def __init__(self, constant):
        super().__init__()
        self.output = constant
    
    def __call__(self):
        return self.output

In [ ]:
%nbdev_export
class Subtract(BaseFunction):
    "A function that subtracts one value from another."
    def __init__(self):
        super().__init__()
    
    def __call__(self):
        self.output = self.links[0].get_output()-self.links[1].get_output()
        return self.output

In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "Proportional function."
    def __init__(self, gain):
        super().__init__()
        self.gain = gain
    
    def __call__(self):
        self.output = input * self.gain
        return self.output

In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "Integration function."
    def __init__(self, gain, slow):
        super().__init__()
        self.gain = gain
        self.slow = slow
    
    def __call__(self):
        input = self.links[0].get_output()
        self.output = self.output +  ((input * self.gain) - self.output)/self.slow
        return self.output
 

In [ ]:
integrator = Integration(3, 10)
integrator.add_link(Constant(5))
output = integrator()
print(output)

[1.5]


In [ ]:
#hide
integrator.set_output(np.array([0]))
output = integrator()
assert output == [1.5]

In [ ]:
%nbdev_export
class PCTNode():
    "A single PCT controller."
    def __init__(self, perception, name="pctnode"):
        self.name=name 
        self.perceptionCollection = [perception]
        reference = Constant(1)
        self.referenceCollection = [reference]
        comparator = Subtract()
        self.comparatorCollection = [comparator]
        self.outputCollection = [Integration(75, 100)]
    
    def __call__(self):
        for perceptionFunction in self.perceptionCollection:
            perceptionFunction()

        for referenceFunction in self.referenceCollection:
            referenceFunction()               
            
        self.comparatorCollection[0].add_link(self.referenceCollection[-1])
        self.comparatorCollection[0].add_link(self.perceptionCollection[-1])

        for comparatorFunction in self.comparatorCollection:
            comparatorFunction()

        self.outputCollection[0].add_link(self.comparatorCollection[-1])
        for outputFunction in self.outputCollection:
            outputFunction()
            
        self.output = self.outputCollection[-1]()
        return self.output
    
    def get_output_function(self):
        return self.outputCollection[-1]
    
    def summary(self):
        print(self.name, type(self).__name__)
        print("----------------------------")


In [ ]:
world = Integration(50, 100)
node = PCTNode(world)
world.add_link(node.get_output_function())
output = node()
print(output[0])
assert abs(output[0]-1.4925) < 0.000001

1.4925000000000002


In [ ]:
node.summary()

pctnode PCTNode
----------------------------


In [ ]:
#hide
from nbdev import *
notebook2script()